In [1]:
#This notebook contains data engineering and data analysis on predicting spread and performace of various ml models

In [2]:
#import packages

import pandas as pd
import numpy as np
import warnings
from sklearn import model_selection, linear_model, preprocessing, ensemble, svm, tree, neighbors
import matplotlib as plt

In [3]:
#load datasets

stadiums = pd.read_csv('../data/nfl_stadiums.csv', encoding='unicode_escape')
teams = pd.read_csv('../data/nfl_teams.csv')
games = pd.read_csv('../data/spreadspoke_scores.csv')
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13783,1/7/2024,2023,18,False,New Orleans Saints,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,Caesars Superdome,False,72.0,0.0,NaN,indoor
13784,1/7/2024,2023,18,False,New York Giants,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,MetLife Stadium,False,NaN,NaN,NaN,NaN
13785,1/7/2024,2023,18,False,San Francisco 49ers,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN
13786,1/7/2024,2023,18,False,Tennessee Titans,NaN,NaN,Jacksonville Jaguars,NaN,NaN,NaN,Nissan Stadium,False,NaN,NaN,NaN,NaN


In [4]:
#add score total and score difference (home - away)

games['score_total'] = games['score_home'] + games['score_away']
games['score_difference'] = games['score_away'] - games['score_home']
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71.0,NaN,37.0,9.0
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70.0,NaN,52.0,-38.0
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82.0,NaN,34.0,-20.0
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78.0,NaN,33.0,5.0
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62.0,NaN,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13783,1/7/2024,2023,18,False,New Orleans Saints,NaN,NaN,Atlanta Falcons,NaN,NaN,NaN,Caesars Superdome,False,72.0,0.0,NaN,indoor,NaN,NaN
13784,1/7/2024,2023,18,False,New York Giants,NaN,NaN,Philadelphia Eagles,NaN,NaN,NaN,MetLife Stadium,False,NaN,NaN,NaN,NaN,NaN,NaN
13785,1/7/2024,2023,18,False,San Francisco 49ers,NaN,NaN,Los Angeles Rams,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN,NaN,NaN
13786,1/7/2024,2023,18,False,Tennessee Titans,NaN,NaN,Jacksonville Jaguars,NaN,NaN,NaN,Nissan Stadium,False,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#remove games without scores, spread, or o/u line

games = games.dropna(subset=['score_home', 'score_away', 'spread_favorite', 'over_under_line'])
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
350,1/14/1968,1967,Superbowl,True,Green Bay Packers,33.0,14.0,Oakland Raiders,GB,-13.5,43,Orange Bowl,True,60.0,12.0,74.0,NaN,47.0,-19.0
538,1/12/1969,1968,Superbowl,True,Baltimore Colts,7.0,16.0,New York Jets,IND,-18.0,40,Orange Bowl,True,66.0,12.0,80.0,rain,23.0,9.0
727,1/11/1970,1969,Superbowl,True,Kansas City Chiefs,23.0,7.0,Minnesota Vikings,MIN,-12.0,39,Tulane Stadium,True,55.0,14.0,84.0,rain,30.0,-16.0
916,1/17/1971,1970,Superbowl,True,Baltimore Colts,16.0,13.0,Dallas Cowboys,IND,-2.5,36,Orange Bowl,True,59.0,11.0,60.0,NaN,29.0,-3.0
1105,1/16/1972,1971,Superbowl,True,Dallas Cowboys,24.0,3.0,Miami Dolphins,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,40.0,NaN,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,Pittsburgh Steelers,10.0,24.0,Arizona Cardinals,PIT,-6.0,41,Acrisure Stadium,False,NaN,NaN,NaN,NaN,34.0,14.0
13705,12/3/2023,2023,13,False,Tampa Bay Buccaneers,14.0,10.0,Carolina Panthers,TB,-3.5,36.5,Raymond James Stadium,False,NaN,NaN,NaN,NaN,24.0,-4.0
13706,12/3/2023,2023,13,False,Tennessee Titans,28.0,31.0,Indianapolis Colts,IND,-1.0,42,Nissan Stadium,False,NaN,NaN,NaN,NaN,59.0,3.0
13707,12/3/2023,2023,13,False,Washington Commanders,15.0,45.0,Miami Dolphins,MIA,-9.0,49.5,FedEx Field,False,NaN,NaN,NaN,NaN,60.0,30.0


In [6]:
#convert team_home and team_away to key values

team_name = teams['team_name']
team_id = teams['team_id']

name_id_dict = {}

for i in range(0, len(team_name)):
    name_id_dict[team_name[i]] = team_id[i] 
name_id_dict

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    games['team_home'] = games['team_home'].replace(name_id_dict)
    games['team_away'] = games['team_away'].replace(name_id_dict)
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
350,1/14/1968,1967,Superbowl,True,GB,33.0,14.0,LVR,GB,-13.5,43,Orange Bowl,True,60.0,12.0,74.0,NaN,47.0,-19.0
538,1/12/1969,1968,Superbowl,True,IND,7.0,16.0,NYJ,IND,-18.0,40,Orange Bowl,True,66.0,12.0,80.0,rain,23.0,9.0
727,1/11/1970,1969,Superbowl,True,KC,23.0,7.0,MIN,MIN,-12.0,39,Tulane Stadium,True,55.0,14.0,84.0,rain,30.0,-16.0
916,1/17/1971,1970,Superbowl,True,IND,16.0,13.0,DAL,IND,-2.5,36,Orange Bowl,True,59.0,11.0,60.0,NaN,29.0,-3.0
1105,1/16/1972,1971,Superbowl,True,DAL,24.0,3.0,MIA,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,40.0,NaN,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,PIT,10.0,24.0,ARI,PIT,-6.0,41,Acrisure Stadium,False,NaN,NaN,NaN,NaN,34.0,14.0
13705,12/3/2023,2023,13,False,TB,14.0,10.0,CAR,TB,-3.5,36.5,Raymond James Stadium,False,NaN,NaN,NaN,NaN,24.0,-4.0
13706,12/3/2023,2023,13,False,TEN,28.0,31.0,IND,IND,-1.0,42,Nissan Stadium,False,NaN,NaN,NaN,NaN,59.0,3.0
13707,12/3/2023,2023,13,False,WAS,15.0,45.0,MIA,MIA,-9.0,49.5,FedEx Field,False,NaN,NaN,NaN,NaN,60.0,30.0


In [7]:
#change spread to always relate to home team

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    games['spread_favorite'] = np.where(games['team_favorite_id'] == games['team_away'], np.abs(games['spread_favorite']), games['spread_favorite'])
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
350,1/14/1968,1967,Superbowl,True,GB,33.0,14.0,LVR,GB,-13.5,43,Orange Bowl,True,60.0,12.0,74.0,NaN,47.0,-19.0
538,1/12/1969,1968,Superbowl,True,IND,7.0,16.0,NYJ,IND,-18.0,40,Orange Bowl,True,66.0,12.0,80.0,rain,23.0,9.0
727,1/11/1970,1969,Superbowl,True,KC,23.0,7.0,MIN,MIN,12.0,39,Tulane Stadium,True,55.0,14.0,84.0,rain,30.0,-16.0
916,1/17/1971,1970,Superbowl,True,IND,16.0,13.0,DAL,IND,-2.5,36,Orange Bowl,True,59.0,11.0,60.0,NaN,29.0,-3.0
1105,1/16/1972,1971,Superbowl,True,DAL,24.0,3.0,MIA,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,40.0,NaN,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,PIT,10.0,24.0,ARI,PIT,-6.0,41,Acrisure Stadium,False,NaN,NaN,NaN,NaN,34.0,14.0
13705,12/3/2023,2023,13,False,TB,14.0,10.0,CAR,TB,-3.5,36.5,Raymond James Stadium,False,NaN,NaN,NaN,NaN,24.0,-4.0
13706,12/3/2023,2023,13,False,TEN,28.0,31.0,IND,IND,1.0,42,Nissan Stadium,False,NaN,NaN,NaN,NaN,59.0,3.0
13707,12/3/2023,2023,13,False,WAS,15.0,45.0,MIA,MIA,9.0,49.5,FedEx Field,False,NaN,NaN,NaN,NaN,60.0,30.0


In [8]:
#change schedule week to fully numeric

schedule_week_dict = {'Wildcard': 19, 'Division': 20, 'Conference': 21, 'Superbowl': 22}

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    games['schedule_week'] = games['schedule_week'].replace(schedule_week_dict)
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
350,1/14/1968,1967,22,True,GB,33.0,14.0,LVR,GB,-13.5,43,Orange Bowl,True,60.0,12.0,74.0,NaN,47.0,-19.0
538,1/12/1969,1968,22,True,IND,7.0,16.0,NYJ,IND,-18.0,40,Orange Bowl,True,66.0,12.0,80.0,rain,23.0,9.0
727,1/11/1970,1969,22,True,KC,23.0,7.0,MIN,MIN,12.0,39,Tulane Stadium,True,55.0,14.0,84.0,rain,30.0,-16.0
916,1/17/1971,1970,22,True,IND,16.0,13.0,DAL,IND,-2.5,36,Orange Bowl,True,59.0,11.0,60.0,NaN,29.0,-3.0
1105,1/16/1972,1971,22,True,DAL,24.0,3.0,MIA,DAL,-6.0,34,Tulane Stadium,True,34.0,18.0,40.0,NaN,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,PIT,10.0,24.0,ARI,PIT,-6.0,41,Acrisure Stadium,False,NaN,NaN,NaN,NaN,34.0,14.0
13705,12/3/2023,2023,13,False,TB,14.0,10.0,CAR,TB,-3.5,36.5,Raymond James Stadium,False,NaN,NaN,NaN,NaN,24.0,-4.0
13706,12/3/2023,2023,13,False,TEN,28.0,31.0,IND,IND,1.0,42,Nissan Stadium,False,NaN,NaN,NaN,NaN,59.0,3.0
13707,12/3/2023,2023,13,False,WAS,15.0,45.0,MIA,MIA,9.0,49.5,FedEx Field,False,NaN,NaN,NaN,NaN,60.0,30.0


In [9]:
#replace null weather values with column average

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    games['weather_temperature'].fillna(games['weather_temperature'].mean(), inplace=True)
    games['weather_wind_mph'].fillna(games['weather_wind_mph'].mean(), inplace=True)
    games['weather_humidity'].fillna(games['weather_humidity'].mean(), inplace=True)
    games['weather_detail'].fillna("N/A", inplace=True)
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,score_total,score_difference
350,1/14/1968,1967,22,True,GB,33.0,14.0,LVR,GB,-13.5,43,Orange Bowl,True,60.00000,12.000000,74.000000,N/A,47.0,-19.0
538,1/12/1969,1968,22,True,IND,7.0,16.0,NYJ,IND,-18.0,40,Orange Bowl,True,66.00000,12.000000,80.000000,rain,23.0,9.0
727,1/11/1970,1969,22,True,KC,23.0,7.0,MIN,MIN,12.0,39,Tulane Stadium,True,55.00000,14.000000,84.000000,rain,30.0,-16.0
916,1/17/1971,1970,22,True,IND,16.0,13.0,DAL,IND,-2.5,36,Orange Bowl,True,59.00000,11.000000,60.000000,N/A,29.0,-3.0
1105,1/16/1972,1971,22,True,DAL,24.0,3.0,MIA,DAL,-6.0,34,Tulane Stadium,True,34.00000,18.000000,40.000000,N/A,27.0,-21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,PIT,10.0,24.0,ARI,PIT,-6.0,41,Acrisure Stadium,False,60.11573,7.084957,66.555189,N/A,34.0,14.0
13705,12/3/2023,2023,13,False,TB,14.0,10.0,CAR,TB,-3.5,36.5,Raymond James Stadium,False,60.11573,7.084957,66.555189,N/A,24.0,-4.0
13706,12/3/2023,2023,13,False,TEN,28.0,31.0,IND,IND,1.0,42,Nissan Stadium,False,60.11573,7.084957,66.555189,N/A,59.0,3.0
13707,12/3/2023,2023,13,False,WAS,15.0,45.0,MIA,MIA,9.0,49.5,FedEx Field,False,60.11573,7.084957,66.555189,N/A,60.0,30.0


In [10]:
#reorder data for clarity

order = ['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff', 'team_home', 'score_home', 'score_away', 'team_away', 'score_total', 'score_difference', 'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium', 'stadium_neutral', 'weather_temperature', 'weather_wind_mph', 'weather_humidity', 'weather_detail']

games = games[order]
games

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,score_total,score_difference,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
350,1/14/1968,1967,22,True,GB,33.0,14.0,LVR,47.0,-19.0,GB,-13.5,43,Orange Bowl,True,60.00000,12.000000,74.000000,N/A
538,1/12/1969,1968,22,True,IND,7.0,16.0,NYJ,23.0,9.0,IND,-18.0,40,Orange Bowl,True,66.00000,12.000000,80.000000,rain
727,1/11/1970,1969,22,True,KC,23.0,7.0,MIN,30.0,-16.0,MIN,12.0,39,Tulane Stadium,True,55.00000,14.000000,84.000000,rain
916,1/17/1971,1970,22,True,IND,16.0,13.0,DAL,29.0,-3.0,IND,-2.5,36,Orange Bowl,True,59.00000,11.000000,60.000000,N/A
1105,1/16/1972,1971,22,True,DAL,24.0,3.0,MIA,27.0,-21.0,DAL,-6.0,34,Tulane Stadium,True,34.00000,18.000000,40.000000,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,12/3/2023,2023,13,False,PIT,10.0,24.0,ARI,34.0,14.0,PIT,-6.0,41,Acrisure Stadium,False,60.11573,7.084957,66.555189,N/A
13705,12/3/2023,2023,13,False,TB,14.0,10.0,CAR,24.0,-4.0,TB,-3.5,36.5,Raymond James Stadium,False,60.11573,7.084957,66.555189,N/A
13706,12/3/2023,2023,13,False,TEN,28.0,31.0,IND,59.0,3.0,IND,1.0,42,Nissan Stadium,False,60.11573,7.084957,66.555189,N/A
13707,12/3/2023,2023,13,False,WAS,15.0,45.0,MIA,60.0,30.0,MIA,9.0,49.5,FedEx Field,False,60.11573,7.084957,66.555189,N/A


In [11]:
#drop unnecessary features for spread prediction

gamesDrop = games.drop(columns=['schedule_date', 'stadium', 'score_home', 'score_away'])
gamesDrop

,schedule_season,schedule_week,schedule_playoff,team_home,team_away,score_total,score_difference,team_favorite_id,spread_favorite,over_under_line,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
350,1967,22,True,GB,LVR,47.0,-19.0,GB,-13.5,43,True,60.00000,12.000000,74.000000,N/A
538,1968,22,True,IND,NYJ,23.0,9.0,IND,-18.0,40,True,66.00000,12.000000,80.000000,rain
727,1969,22,True,KC,MIN,30.0,-16.0,MIN,12.0,39,True,55.00000,14.000000,84.000000,rain
916,1970,22,True,IND,DAL,29.0,-3.0,IND,-2.5,36,True,59.00000,11.000000,60.000000,N/A
1105,1971,22,True,DAL,MIA,27.0,-21.0,DAL,-6.0,34,True,34.00000,18.000000,40.000000,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13704,2023,13,False,PIT,ARI,34.0,14.0,PIT,-6.0,41,False,60.11573,7.084957,66.555189,N/A
13705,2023,13,False,TB,CAR,24.0,-4.0,TB,-3.5,36.5,False,60.11573,7.084957,66.555189,N/A
13706,2023,13,False,TEN,IND,59.0,3.0,IND,1.0,42,False,60.11573,7.084957,66.555189,N/A
13707,2023,13,False,WAS,MIA,60.0,30.0,MIA,9.0,49.5,False,60.11573,7.084957,66.555189,N/A


In [12]:
#Filter for 2023 season only

games23 = gamesDrop[gamesDrop['schedule_season'] == 2023]
games23 = games23.drop(columns=['schedule_season', 'schedule_playoff', 'team_favorite_id'])
games23['schedule_week'] = games23['schedule_week'].astype(int)
#games23['over_under_line'] = games23['over_under_line'].replace('[^\d.]', '', regex=True).astype(float)
games23['over_under_line'] = games23['over_under_line'].astype(float)
games23

,schedule_week,team_home,team_away,score_total,score_difference,spread_favorite,over_under_line,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13516,1,KC,DET,41.0,1.0,-4.5,53.0,False,60.11573,7.084957,66.555189,N/A
13517,1,ATL,CAR,34.0,-14.0,-3.5,39.5,False,72.00000,0.000000,66.555189,indoor
13518,1,BAL,HOU,34.0,-16.0,-9.5,43.5,False,60.11573,7.084957,66.555189,N/A
13519,1,CHI,GB,58.0,18.0,-1.0,41.5,False,60.11573,7.084957,66.555189,N/A
13520,1,CLE,CIN,27.0,-21.0,2.0,47.5,False,60.11573,7.084957,66.555189,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
13704,13,PIT,ARI,34.0,14.0,-6.0,41.0,False,60.11573,7.084957,66.555189,N/A
13705,13,TB,CAR,24.0,-4.0,-3.5,36.5,False,60.11573,7.084957,66.555189,N/A
13706,13,TEN,IND,59.0,3.0,1.0,42.0,False,60.11573,7.084957,66.555189,N/A
13707,13,WAS,MIA,60.0,30.0,9.0,49.5,False,60.11573,7.084957,66.555189,N/A


In [13]:
# Linear Regression for Spread (KFold random test/train)

X = pd.get_dummies(games23, columns=['team_home', 'team_away', 'weather_detail'])
X = X.drop(['score_difference', 'score_total'], axis=1)
Y = games23['score_difference']
regr = linear_model.LinearRegression()

r2, mse = [], []

kf = model_selection.KFold(n_splits=5, shuffle=True)
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    
    regr.fit(X_train, Y_train)
    
    mse += [np.mean((regr.predict(X_test) - Y_test) ** 2)]
    r2 += [regr.score(X_test, Y_test)]

print("Spread:")
print("mean Squared Error = {:.4f} ±{:.4f}".format(np.mean(mse), np.std(mse)))
print("R^2        = {:.4f} ±{:.4f}".format(np.mean(r2), np.std(r2)))

Spread:
mean Squared Error = 248.0029 ±25.0504
R^2        = -0.3249 ±0.3262


In [14]:
#Linear Regression for Spread (Week 13 Only)

X = pd.get_dummies(games23, columns=['team_home', 'team_away', 'weather_detail'])
X = X.drop(['score_difference', 'score_total'], axis=1)
Y = games23['score_difference']
regr = linear_model.LinearRegression()

X_train = X[games23['schedule_week'].isin(range(1, 13))]
Y_train = Y[games23['schedule_week'].isin(range(1, 13))]
X_test = X[games23['schedule_week'] == 13]
Y_test = Y[games23['schedule_week'] == 13]

regr.fit(X_train, Y_train)
predictions = regr.predict(X_test)

print("mean Squared Error = {:.4f}".format(np.mean((predictions - Y_test) ** 2)))
print("R^2        = {:.4f}".format(regr.score(X_test, Y_test)))

mean Squared Error = 187.9615
R^2        = -0.2376


In [15]:
#Create dataframe to display

week13Results = pd.DataFrame({
    'Vegas_Spread': games23.loc[X_test.index, 'spread_favorite'],
    'Predicted_Spread': predictions,
    'Actual_Spread': Y_test,
    'Home Team': games23.loc[X_test.index, 'team_home'],
    'Away Team': games23.loc[X_test.index, 'team_away']
})

In [16]:
#Cacluate result of placing bet based on vegas and predicted spread

results = []

for i in range(0, len(week13Results)):
    if (week13Results.iloc[i]['Predicted_Spread'] < week13Results.iloc[i]['Vegas_Spread']):
        
        if ((week13Results.iloc[i]['Actual_Spread'] - week13Results.iloc[i]['Vegas_Spread']) < 0):
            results += ['W']
        elif ((week13Results.iloc[i]['Actual_Spread'] - week13Results.iloc[i]['Vegas_Spread']) > 0):
            results += ['L']
        else:
            results += ['Void']
            
    elif (week13Results.iloc[i]['Predicted_Spread'] > week13Results.iloc[i]['Vegas_Spread']):
        
        if ((week13Results.iloc[i]['Actual_Spread'] - week13Results.iloc[i]['Vegas_Spread']) > 0):
            results += ['W']
        elif ((week13Results.iloc[i]['Actual_Spread'] - week13Results.iloc[i]['Vegas_Spread']) < 0):
            results += ['L']
        else:
            results += ['Void']
            
    else:
        results += ['Void']

In [17]:
week13Results['Result'] = results
week13Results

,Vegas_Spread,Predicted_Spread,Actual_Spread,Home Team,Away Team,Result
13696,-9.5,-26.50,-6.0,DAL,SEA,L
13697,6.0,2.25,-8.0,GB,KC,W
13698,-3.0,-5.75,-5.0,HOU,DEN,W
13699,-3.5,2.75,-17.0,LAR,CLE,L
13700,4.5,7.00,6.0,NE,LAC,W
13701,4.0,9.75,5.0,NO,DET,W
13702,2.0,-2.50,5.0,NYJ,ATL,L
13703,3.0,6.25,23.0,PHI,SF,W
13704,-6.0,-16.00,14.0,PIT,ARI,L
13705,-3.5,-4.00,-4.0,TB,CAR,W


In [18]:
#Testing for 2023

X = pd.get_dummies(games23, columns=['team_home', 'team_away', 'weather_detail'])
X = X.drop(['score_difference', 'score_total'], axis=1)
Y = games23['score_difference']
regr = linear_model.LinearRegression()

X_train = X[games23['schedule_week'].isin(range(1, 13))] #change week and training data
Y_train = Y[games23['schedule_week'].isin(range(1, 13))]
X_test = X[games23['schedule_week'] == 13]
Y_test = Y[games23['schedule_week'] == 13]

regr.fit(X_train, Y_train)
predictions = regr.predict(X_test)

print("mean Squared Error = {:.4f}".format(np.mean((predictions - Y_test) ** 2)))
print("R^2        = {:.4f}".format(regr.score(X_test, Y_test)))

result_df = pd.DataFrame({
    'Vegas_Spread': games23.loc[X_test.index, 'spread_favorite'],
    'Predicted_Spread': predictions,
    'Actual_Spread': Y_test,
    'Home Team': games23.loc[X_test.index, 'team_home'],
    'Away Team': games23.loc[X_test.index, 'team_away']
})

results = []

for i in range(0, len(result_df)):
    if (result_df.iloc[i]['Predicted_Spread'] < result_df.iloc[i]['Vegas_Spread']):
        
        if ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) < 0):
            results += ['W']
        elif ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) > 0):
            results += ['L']
        else:
            results += ['Void']
            
    elif (result_df.iloc[i]['Predicted_Spread'] > result_df.iloc[i]['Vegas_Spread']):
        
        if ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) > 0):
            results += ['W']
        elif ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) < 0):
            results += ['L']
        else:
            results += ['Void']
            
    else:
        results += ['Void']
        
result_df['Result'] = results
print(len(result_df[result_df['Result'] == 'W']))
result_df

mean Squared Error = 187.9615
R^2        = -0.2376
8


,Vegas_Spread,Predicted_Spread,Actual_Spread,Home Team,Away Team,Result
13696,-9.5,-26.50,-6.0,DAL,SEA,L
13697,6.0,2.25,-8.0,GB,KC,W
13698,-3.0,-5.75,-5.0,HOU,DEN,W
13699,-3.5,2.75,-17.0,LAR,CLE,L
13700,4.5,7.00,6.0,NE,LAC,W
13701,4.0,9.75,5.0,NO,DET,W
13702,2.0,-2.50,5.0,NYJ,ATL,L
13703,3.0,6.25,23.0,PHI,SF,W
13704,-6.0,-16.00,14.0,PIT,ARI,L
13705,-3.5,-4.00,-4.0,TB,CAR,W


In [19]:
#Function for generating above results for variable regressor and week

def spread(regressor, week):
    X = pd.get_dummies(games23, columns=['team_home', 'team_away', 'weather_detail'])
    X = X.drop(['score_difference', 'score_total'], axis=1)
    Y = games23['score_difference']
    regr = regressor

    X_train = X[games23['schedule_week'].isin(range(1, week))] 
    Y_train = Y[games23['schedule_week'].isin(range(1, week))]
    X_test = X[games23['schedule_week'] == week]
    Y_test = Y[games23['schedule_week'] == week]

    regr.fit(X_train, Y_train)
    predictions = regr.predict(X_test)

    result_df = pd.DataFrame({
        'Vegas_Spread': games23.loc[X_test.index, 'spread_favorite'],
        'Predicted_Spread': predictions,
        'Actual_Spread': Y_test,
        'Home Team': games23.loc[X_test.index, 'team_home'],
        'Away Team': games23.loc[X_test.index, 'team_away']
    })

    results = []

    for i in range(0, len(result_df)):
        if (result_df.iloc[i]['Predicted_Spread'] < result_df.iloc[i]['Vegas_Spread']):

            if ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) < 0):
                results += ['W']
            elif ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) > 0):
                results += ['L']
            else:
                results += ['Void']

        elif (result_df.iloc[i]['Predicted_Spread'] > result_df.iloc[i]['Vegas_Spread']):

            if ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) > 0):
                results += ['W']
            elif ((result_df.iloc[i]['Actual_Spread'] - result_df.iloc[i]['Vegas_Spread']) < 0):
                results += ['L']
            else:
                results += ['Void']

        else:
            results += ['Void']

    result_df['Result'] = results
    
    return [len(result_df[result_df['Result'] == 'W']), len(result_df[result_df['Result'] == 'L']), len(result_df[result_df['Result'] == 'Void'])]

In [20]:
#Create dataset for spread profit since 2000 for different regressors

df = pd.DataFrame()
regressors = [linear_model.LinearRegression(), tree.DecisionTreeRegressor(), svm.SVR(), 
              ensemble.RandomForestRegressor(n_estimators=100), ensemble.GradientBoostingRegressor(), 
              neighbors.KNeighborsRegressor(n_neighbors=7, weights='uniform'), linear_model.Lasso(), 
              linear_model.Ridge(), linear_model.ElasticNet()]
years = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 
         2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

for r in regressors:
    
    totalWins = 0
    winsList = []
    totalLosses = 0
    lossesList = []
    totalTies = 0
    tiesList = []
    
    for year in years:

        games23 = gamesDrop[gamesDrop['schedule_season'] == year]
        games23 = games23.drop(columns=['schedule_season', 'schedule_playoff', 'team_favorite_id'])
        games23['schedule_week'] = games23['schedule_week'].astype(int)
        games23['over_under_line'] = games23['over_under_line'].replace('[^\d.]', '', regex=True).astype(float)

        wins = 0
        losses = 0
        ties = 0

        if (year < 2021):

            for i in range(4, 18):
                x = spread(r,i)
                wins += x[0]
                losses += x[1]
                ties += x[2]

            for i in range(19, 23):
                x = spread(r,i)
                wins += x[0]
                losses += x[1]
                ties += x[2]

        elif (year < 2023):

            for i in range(4, 23):
                x = spread(r,i)
                wins += x[0]
                losses += x[1]
                ties += x[2]

        else:

            for i in range(4, 14):
                x = spread(r,i)
                wins += x[0]
                losses += x[1]
                ties += x[2]

        totalWins += wins
        winsList += [wins]
        totalLosses += losses
        lossesList += [losses]
        totalTies += ties
        tiesList += [ties]

    winsList += [totalWins]
    lossesList += [totalLosses]
    tiesList += [totalTies]
    
    result_df = pd.DataFrame({
        'Year': [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 
                 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 'Total'],
        'Wins': winsList,
        'Losses': lossesList,
        'Ties': tiesList,
        'Regressor': [r]*25})
    
    result_df['Profit'] = result_df['Wins'] - (1.1 * result_df['Losses'])
    
    df = pd.concat([df, result_df])
    
df

,Year,Wins,Losses,Ties,Regressor,Profit
0,2000,109,102,4,LinearRegression(),-3.2
1,2001,90,116,10,LinearRegression(),-37.6
2,2002,115,102,4,LinearRegression(),2.8
3,2003,97,117,7,LinearRegression(),-31.7
4,2004,111,105,5,LinearRegression(),-4.5
...,...,...,...,...,...,...
20,2020,107,109,5,ElasticNet(),-12.9
21,2021,124,109,4,ElasticNet(),4.1
22,2022,112,118,6,ElasticNet(),-17.8
23,2023,72,68,5,ElasticNet(),-2.8


In [21]:
#Highest performing regression algorithm this century

df[df['Year'] == 'Total'].sort_values(by='Profit', ascending=False)

,Year,Wins,Losses,Ties,Regressor,Profit
24,Total,2560,2491,173,KNeighborsRegressor(n_neighbors=7),-180.1
24,Total,2575,2510,139,SVR(),-186.0
24,Total,2570,2515,139,ElasticNet(),-196.5
24,Total,2554,2531,139,Lasso(),-230.1
24,Total,2548,2532,144,LinearRegression(),-237.2
24,Total,2448,2490,286,DecisionTreeRegressor(),-291.0
24,Total,2516,2567,141,"(DecisionTreeRegressor(max_features='auto', ra...",-307.7
24,Total,2507,2578,139,([DecisionTreeRegressor(criterion='friedman_ms...,-328.8
24,Total,2500,2585,139,Ridge(),-343.5


In [22]:
#Highest performing algorithm, season combination

df.sort_values(by='Profit', ascending=False)

,Year,Wins,Losses,Ties,Regressor,Profit
23,2023,88,52,5,LinearRegression(),30.8
23,2023,82,51,12,DecisionTreeRegressor(),25.9
7,2007,125,92,2,([DecisionTreeRegressor(criterion='friedman_ms...,23.8
12,2012,123,93,3,KNeighborsRegressor(n_neighbors=7),20.7
6,2006,122,94,5,Lasso(),18.6
...,...,...,...,...,...,...
24,Total,2548,2532,144,LinearRegression(),-237.2
24,Total,2448,2490,286,DecisionTreeRegressor(),-291.0
24,Total,2516,2567,141,"(DecisionTreeRegressor(max_features='auto', ra...",-307.7
24,Total,2507,2578,139,([DecisionTreeRegressor(criterion='friedman_ms...,-328.8
